<a href="https://colab.research.google.com/github/toftaker/jernbaneverket_iv/blob/master/kalman_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gpxpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00


In [5]:
# prompt: extract position and time from a gpx file

import gpxpy
import gpxpy.gpx

def extract_position_time(gpx_file_path):
    """Extracts position (latitude, longitude) and time from a GPX file.

    Args:
        gpx_file_path (str): The path to the GPX file.

    Returns:
        list: A list of dictionaries, where each dictionary contains the latitude,
              longitude, and time of a track point. Returns an empty list if
              the file is invalid or no track points are found.
    """
    try:
        with open(gpx_file_path, 'r') as gpx_file:
            gpx = gpxpy.parse(gpx_file)
    except FileNotFoundError:
        print(f"Error: File not found at {gpx_file_path}")
        return []
    except Exception as e:  # Catch other potential errors during parsing
        print(f"Error parsing GPX file: {e}")
        return []

    track_points = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                track_points.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'time': point.time
                })
    return track_points


# Example usage (replace with your GPX file path)
gpx_file = "gpxdata/Bråtesten_22_22.gpx"
extracted_data = extract_position_time(gpx_file)

#if extracted_data:
#    for point in extracted_data:
#        print(f"Latitude: {point['latitude']}, Longitude: {point['longitude']}, Time: {point['time']}")
#else:
#    print("No data extracted from the GPX file.")


In [10]:
import pandas as pd
df = pd.DataFrame(extracted_data)
df

,latitude,longitude,time
0,63.420655,10.339240,2024-09-26 16:26:00+00:00
1,63.420720,10.339225,2024-09-26 16:26:03+00:00
2,63.420778,10.339132,2024-09-26 16:26:05+00:00
3,63.420846,10.339208,2024-09-26 16:26:07+00:00
4,63.420868,10.339273,2024-09-26 16:26:08+00:00
...,...,...,...
580,63.417936,10.279823,2024-09-26 16:48:21+00:00
581,63.417906,10.279685,2024-09-26 16:48:23+00:00
582,63.417881,10.279524,2024-09-26 16:48:25+00:00
583,63.417878,10.279440,2024-09-26 16:48:26+00:00


In [14]:
import numpy as np
gps_data = np.array(df[['latitude','longitude']])
gps_data

array([[63.420655, 10.33924 ],
       [63.42072 , 10.339225],
       [63.420778, 10.339132],
       ...,
       [63.417881, 10.279524],
       [63.417878, 10.27944 ],
       [63.41788 , 10.279311]])

In [29]:
# prompt: Write code to filter a gps data using a kalman filter

import numpy as np

class KalmanFilter:
    def __init__(self, initial_state, initial_covariance, process_noise, measurement_noise):
        self.state = initial_state
        self.covariance = initial_covariance
        self.process_noise = process_noise
        self.measurement_noise = measurement_noise

    def predict(self, motion_model):
        # Predict the next state based on the motion model
        self.state = np.dot(motion_model, self.state)
        self.covariance = np.dot(np.dot(motion_model, self.covariance), motion_model.T) + self.process_noise
        return self.state

    def update(self, measurement, measurement_matrix):
        # Update the state based on the measurement
        innovation = measurement - np.dot(measurement_matrix, self.state)
        innovation_covariance = np.dot(np.dot(measurement_matrix, self.covariance), measurement_matrix.T) + self.measurement_noise
        kalman_gain = np.dot(self.covariance, np.dot(measurement_matrix.T, np.linalg.inv(innovation_covariance)))
        self.state = self.state + np.dot(kalman_gain, innovation)
        self.covariance = np.dot((np.eye(len(self.state)) - np.dot(kalman_gain, measurement_matrix)), self.covariance)
        return self.state

# Example GPS data (latitude, longitude)
#gps_data = np.array([
#    [37.7749, -122.4194],
#    [37.7750, -122.4195],
#    [37.7752, -122.4193],
#    [37.7755, -122.4198],
#    [37.7753, -122.4200], # Example of a potential outlier
#])


# Initialize Kalman Filter
initial_state = np.array([gps_data[0, 0], gps_data[0, 1], 0, 0]) # Initial state: latitude, longitude, velocity in latitude, velocity in longitude
initial_covariance = np.eye(4) * 0.1  # Initial uncertainty
process_noise = np.eye(4) * 0.01 # Process Noise
measurement_noise = np.eye(2) * 1 ##0.01 # Measurement noise

kf = KalmanFilter(initial_state, initial_covariance, process_noise, measurement_noise)

# Motion Model (Assuming constant velocity)
motion_model = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Measurement Matrix (We observe latitude and longitude)
measurement_matrix = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

filtered_gps = []

# Filtering loop
for measurement in gps_data:
    predicted_state = kf.predict(motion_model)
    filtered_state = kf.update(measurement, measurement_matrix)
    filtered_gps.append(filtered_state[:2])  # Append latitude and longitude

filtered_gps = np.array(filtered_gps)

print("Original GPS Data:")
print(gps_data)
print("\nFiltered GPS Data:")
filtered_gps


Original GPS Data:
[[63.420655 10.33924 ]
 [63.42072  10.339225]
 [63.420778 10.339132]
 ...
 [63.417881 10.279524]
 [63.417878 10.27944 ]
 [63.41788  10.279311]]

Filtered GPS Data:


array([[63.420655  , 10.33924   ],
       [63.42067519, 10.33923534],
       [63.42072115, 10.33919299],
       ...,
       [63.4178674 , 10.27955761],
       [63.4178495 , 10.27944176],
       [63.4178412 , 10.27932092]])

In [30]:
import folium
m = folium.Map(location=[63.4, 10.3], zoom_start=12)

folium.PolyLine(
    locations=filtered_gps,
    color="#FF0000",
    weight=5,
    tooltip="Filtrert rute",
).add_to(m)

folium.PolyLine(
    locations=gps_data,
    color="#FFFF00",
    weight=3,
    tooltip="Original rute",
).add_to(m)

m